In [2]:
# !pip install tensorflow yfinance scikit-learn pandas numpy

In [16]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import precision_score

In [17]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import precision_score

# Fetching data from Yahoo Finance
sp500 = yf.Ticker("SPY").history(period="max")

# Preprocessing
sp500.drop(["Dividends", "Stock Splits"], axis=1, inplace=True)
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500 = sp500.loc["1990-01-01":].copy()

# Define additional predictors based on horizons
horizons = [2, 5, 60, 250, 1000]
new_predictors = ["Close", "Volume", "Open", "High", "Low"]  # Original features

for horizon in horizons:
    rolling_average = sp500.rolling(horizon).mean()
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_average["Close"]
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    new_predictors += [ratio_column, trend_column]

sp500 = sp500.dropna()

# Scaling the features
scaler = StandardScaler()
sp500_scaled = sp500.copy()
sp500_scaled[new_predictors] = scaler.fit_transform(sp500[new_predictors])

# Neural Network Model Definition
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Backtest Function
def backtest_nn(data, create_model, predictors, start=2500, step=250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()

        X_train = train[predictors]
        X_test = test[predictors]
        y_train = train["Target"]
        y_test = test["Target"]

        model = create_model(X_train.shape[1])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        preds = model.predict(X_test)
        preds = (preds >= 0.6).astype(int).flatten()
        predictions = pd.Series(preds, index=test.index, name="Predictions")
        combined = pd.concat([test["Target"], predictions], axis=1)

        all_predictions.append(combined)
    return pd.concat(all_predictions)

# Running the Backtest
predictions_nn = backtest_nn(sp500_scaled, create_model, new_predictors)
print(predictions_nn)

# Calculating Precision
precision = precision_score(predictions_nn["Target"], predictions_nn["Predictions"])
print(f"Precision: {precision}")

2/2 [==============================] - 0s 2ms/step
                           Target  Predictions
Date                                          
2006-12-19 00:00:00-05:00       0            0
2006-12-20 00:00:00-05:00       0            0
2006-12-21 00:00:00-05:00       0            0
2006-12-22 00:00:00-05:00       1            1
2006-12-26 00:00:00-05:00       1            0
...                           ...          ...
2024-01-19 00:00:00-05:00       1            0
2024-01-22 00:00:00-05:00       1            0
2024-01-23 00:00:00-05:00       1            0
2024-01-24 00:00:00-05:00       1            0
2024-01-25 00:00:00-05:00       0            0

[4303 rows x 2 columns]
Precision: 0.5473289597000938


In [18]:
predictions_nn

,Target,Predictions
Date,,
2006-12-19 00:00:00-05:00,0,0
2006-12-20 00:00:00-05:00,0,0
2006-12-21 00:00:00-05:00,0,0
2006-12-22 00:00:00-05:00,1,1
2006-12-26 00:00:00-05:00,1,0
...,...,...
2024-01-19 00:00:00-05:00,1,0
2024-01-22 00:00:00-05:00,1,0
2024-01-23 00:00:00-05:00,1,0


In [35]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import precision_score

# Fetching data from Yahoo Finance
sp500 = yf.Ticker("SPY").history(period="max")

# Preprocessing
sp500.drop(["Dividends", "Stock Splits"], axis=1, inplace=True)
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500 = sp500.loc["1990-01-01":].copy()

# Define additional predictors based on horizons
horizons = [2, 5, 60, 250, 1000]
new_predictors = ["Close", "Volume", "Open", "High", "Low"]  # Original features

for horizon in horizons:
    rolling_average = sp500.rolling(horizon).mean()
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_average["Close"]
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    new_predictors += [ratio_column, trend_column]

sp500 = sp500.dropna()

# Scaling the features
scaler = StandardScaler()
sp500_scaled = sp500.copy()
sp500_scaled[new_predictors] = scaler.fit_transform(sp500[new_predictors])

def create_model_tanh(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='tanh', input_shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Backtest Function
def backtest_nn(data, create_model_tanh, predictors, start=2500, step=250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()

        X_train = train[predictors]
        X_test = test[predictors]
        y_train = train["Target"]
        y_test = test["Target"]

        model = create_model(X_train.shape[1])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        preds = model.predict(X_test)
        preds = (preds >= 0.6).astype(int).flatten()
        predictions = pd.Series(preds, index=test.index, name="Predictions")
        combined = pd.concat([test["Target"], predictions], axis=1)

        all_predictions.append(combined)
    return pd.concat(all_predictions)

# Running the Backtest
predictions_nn = backtest_nn(sp500_scaled, create_model_tanh, new_predictors)
print(predictions_nn)

# Calculating Precision
precision = precision_score(predictions_nn["Target"], predictions_nn["Predictions"])
print(f"Precision: {precision}")

2/2 [==============================] - 0s 2ms/step
                           Target  Predictions
Date                                          
2006-12-19 00:00:00-05:00       0            1
2006-12-20 00:00:00-05:00       0            1
2006-12-21 00:00:00-05:00       0            0
2006-12-22 00:00:00-05:00       1            0
2006-12-26 00:00:00-05:00       1            0
2006-12-27 00:00:00-05:00       0            1
2006-12-28 00:00:00-05:00       0            0
2006-12-29 00:00:00-05:00       0            0
2007-01-03 00:00:00-05:00       1            0
2007-01-04 00:00:00-05:00       0            1
2007-01-05 00:00:00-05:00       1            0
2007-01-08 00:00:00-05:00       0            0
2007-01-09 00:00:00-05:00       1            1
2007-01-10 00:00:00-05:00       1            1
2007-01-11 00:00:00-05:00       1            1
2007-01-12 00:00:00-05:00       0            1
2007-01-16 00:00:00-05:00       1            0
2007-01-17 00:00:00-05:00       0            0
2007-01-1

In [41]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import precision_score

# Fetching data from Yahoo Finance
sp500 = yf.Ticker("^GSPC").history(period="max")

# Preprocessing
sp500.drop(["Dividends", "Stock Splits"], axis=1, inplace=True)
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500 = sp500.loc["1990-01-01":].copy()

# Define additional predictors based on horizons
horizons = [2, 5, 60, 250, 1000]
new_predictors = ["Close", "Volume", "Open", "High", "Low"]  # Original features

for horizon in horizons:
    rolling_average = sp500.rolling(horizon).mean()
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_average["Close"]
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    new_predictors += [ratio_column, trend_column]

sp500 = sp500.dropna()

# Scaling the features
scaler = StandardScaler()
sp500_scaled = sp500.copy()
sp500_scaled[new_predictors] = scaler.fit_transform(sp500[new_predictors])

def create_model_sigmoid(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='sigmoid', input_shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation='sigmoid'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Backtest Function
def backtest_nn(data, create_model_sigmoid, predictors, start=2500, step=250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()

        X_train = train[predictors]
        X_test = test[predictors]
        y_train = train["Target"]
        y_test = test["Target"]

        model = create_model(X_train.shape[1])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        preds = model.predict(X_test)
        preds = (preds >= 0.6).astype(int).flatten()
        predictions = pd.Series(preds, index=test.index, name="Predictions")
        combined = pd.concat([test["Target"], predictions], axis=1)

        all_predictions.append(combined)
    return pd.concat(all_predictions)

# Running the Backtest
predictions_nn = backtest_nn(sp500_scaled, create_model, new_predictors)
print(predictions_nn)

# Calculating Precision
precision = precision_score(predictions_nn["Target"], predictions_nn["Predictions"])
print(f"Precision: {precision}")

3/3 [==============================] - 0s 1ms/step
                           Target  Predictions
Date                                          
2003-11-14 00:00:00-05:00       0            1
2003-11-17 00:00:00-05:00       0            1
2003-11-18 00:00:00-05:00       1            1
2003-11-19 00:00:00-05:00       0            0
2003-11-20 00:00:00-05:00       1            0
...                           ...          ...
2024-01-16 00:00:00-05:00       0            0
2024-01-17 00:00:00-05:00       1            1
2024-01-18 00:00:00-05:00       1            0
2024-01-19 00:00:00-05:00       1            0
2024-01-22 00:00:00-05:00       1            0

[5079 rows x 2 columns]
Precision: 0.5559701492537313


In [4]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import precision_score

# Fetching data from Yahoo Finance
sp500 = yf.Ticker("^GSPC").history(period="max")

# Preprocessing
sp500.drop(["Dividends", "Stock Splits"], axis=1, inplace=True)
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500 = sp500.loc["1990-01-01":].copy()

# Define additional predictors based on horizons
horizons = [2, 5, 60, 250, 1000]
new_predictors = ["Close", "Volume", "Open", "High", "Low"]  # Original features

for horizon in horizons:
    rolling_average = sp500.rolling(horizon).mean()
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_average["Close"]
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    new_predictors += [ratio_column, trend_column]

sp500 = sp500.dropna()

# Scaling the features
scaler = StandardScaler()
sp500_scaled = sp500.copy()
sp500_scaled[new_predictors] = scaler.fit_transform(sp500[new_predictors])

def create_model_leaky_relu(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Backtest Function
def backtest_nn(data, create_model_leaky_relu, predictors, start=2500, step=250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()

        X_train = train[predictors]
        X_test = test[predictors]
        y_train = train["Target"]
        y_test = test["Target"]

        model = create_model(X_train.shape[1])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        preds = model.predict(X_test)
        preds = (preds >= 0.6).astype(int).flatten()
        predictions = pd.Series(preds, index=test.index, name="Predictions")
        combined = pd.concat([test["Target"], predictions], axis=1)

        all_predictions.append(combined)
    return pd.concat(all_predictions)

# Running the Backtest
predictions_nn = backtest_nn(sp500_scaled, create_model, new_predictors)
print(predictions_nn)

# Calculating Precision
precision = precision_score(predictions_nn["Target"], predictions_nn["Predictions"])
print(f"Precision: {precision}")

NameError: name 'create_model' is not defined

In [43]:
# Reset the display option to the default value (if needed)
pd.reset_option('display.max_rows')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [10]:
print(sp500.head())

                                 Open        High         Low       Close  \
Date                                                                        
1993-12-14 00:00:00-05:00  465.730011  466.119995  462.459991  463.059998   
1993-12-15 00:00:00-05:00  463.059998  463.690002  461.839996  461.839996   
1993-12-16 00:00:00-05:00  461.859985  463.980011  461.859985  463.339996   
1993-12-17 00:00:00-05:00  463.339996  466.380005  463.339996  466.380005   
1993-12-20 00:00:00-05:00  466.380005  466.899994  465.529999  465.850006   

                              Volume    Tomorrow  Target  Close_Ratio_2  \
Date                                                                      
1993-12-14 00:00:00-05:00  275050000  461.839996       0       0.997157   
1993-12-15 00:00:00-05:00  331770000  463.339996       1       0.998681   
1993-12-16 00:00:00-05:00  284620000  466.380005       1       1.001621   
1993-12-17 00:00:00-05:00  363750000  465.850006       0       1.003270   
1993-12-20

In [19]:
sentiment_data = pd.read_csv('sentiment_data.csv', encoding='ISO-8859-1')
print(sentiment_data.head())

                  date                                              title  \
0  2024-01-24 13:26:06  Stocks May Pause: 5 Reasons to be "Strategical...   
1  2024-01-24 02:59:00  Billionaires Can't Get Enough of This ETF in 2...   
2  2024-01-23 12:02:03  ETFs in Focus as S&P 500 Confirms Bull Market:...   
3  2024-01-23 06:07:14  Secondary squeeze: exploring 3 highly shorted ...   
4  2024-01-22 18:22:37  Why Parametric's Equity Income ETF Looks Compe...   

       positive   neutral      negative  
0  5.955281e-04  0.999267  1.375334e-04  
1  2.126579e-07  1.000000  2.185417e-07  
2  9.697534e-04  0.983527  1.550349e-02  
3  3.383975e-05  0.999961  4.917491e-06  
4  1.563262e-02  0.464915  5.194526e-01  


In [1]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import precision_score

In [9]:
sentiment_data = pd.read_csv('sentiment_data.csv', encoding='ISO-8859-1')
print(sentiment_data.head())

                  date                                              title  \
0  2024-01-24 13:26:06  Stocks May Pause: 5 Reasons to be "Strategical...   
1  2024-01-24 02:59:00  Billionaires Can't Get Enough of This ETF in 2...   
2  2024-01-23 12:02:03  ETFs in Focus as S&P 500 Confirms Bull Market:...   
3  2024-01-23 06:07:14  Secondary squeeze: exploring 3 highly shorted ...   
4  2024-01-22 18:22:37  Why Parametric's Equity Income ETF Looks Compe...   

       positive   neutral      negative  
0  5.955281e-04  0.999267  1.375334e-04  
1  2.126579e-07  1.000000  2.185417e-07  
2  9.697534e-04  0.983527  1.550349e-02  
3  3.383975e-05  0.999961  4.917491e-06  
4  1.563262e-02  0.464915  5.194526e-01  


In [10]:
# Fetching data from Yahoo Finance
sp500 = yf.Ticker("SPY").history(period="max")

In [20]:
# Reset the index to convert 'Date' from index to column
sp500.reset_index(inplace=True)

# Convert 'Date' in sp500 to date format
sp500['Date'] = pd.to_datetime(sp500['Date']).dt.date

# Convert 'date' in sentiment_data to date format
sentiment_data['date'] = pd.to_datetime(sentiment_data['date']).dt.date

# Sort the sentiment_data by 'date'
sentiment_data = sentiment_data.sort_values(by='date')

# Merge S&P 500 data with Sentiment Data
combined_data = pd.merge(sp500, sentiment_data, left_on='Date', right_on='date', how='left')

In [21]:
combined_data

,Date,Open,High,Low,Close,Volume,Capital Gains,Tomorrow,Target,Close_Ratio_2,...,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000,date,title,positive,neutral,negative
0,1997-01-14,47.612693,48.049240,47.496280,47.787312,2111200,0.0,47.670891,0,1.006230,...,36.0,1.147327,135.0,1.494512,527.0,NaN,NaN,NaN,NaN,NaN
1,1997-01-15,47.632087,47.932819,47.418664,47.670891,1583900,0.0,47.864925,1,0.998780,...,35.0,1.143392,134.0,1.489816,526.0,NaN,NaN,NaN,NaN,NaN
2,1997-01-16,47.826121,47.991039,47.496286,47.864925,1308400,0.0,48.155952,1,1.002031,...,36.0,1.146884,134.0,1.494815,526.0,NaN,NaN,NaN,NaN,NaN
3,1997-01-17,47.932829,48.272365,47.874622,48.155952,1604000,0.0,48.214153,1,1.003031,...,37.0,1.152668,134.0,1.502832,526.0,NaN,NaN,NaN,NaN,NaN
4,1997-01-20,48.272360,48.485782,48.097741,48.214153,1889900,0.0,48.602196,1,1.000604,...,37.0,1.152899,134.0,1.503580,526.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6861,2024-01-23,484.010010,485.109985,482.890015,484.859985,49945300,0.0,485.390015,1,1.001456,...,40.0,1.130057,140.0,1.252861,537.0,2024-01-23,Secondary squeeze: exploring 3 highly shorted ...,3.383975e-05,0.999961,4.917491e-06
6862,2024-01-23,484.010010,485.109985,482.890015,484.859985,49945300,0.0,485.390015,1,1.001456,...,40.0,1.130057,140.0,1.252861,537.0,2024-01-23,ETFs in Focus as S&P 500 Confirms Bull Market:...,9.697534e-04,0.983527,1.550349e-02
6863,2024-01-24,487.809998,488.769989,484.880005,485.390015,81765000,0.0,488.029999,1,1.000546,...,41.0,1.130333,140.0,1.253644,537.0,2024-01-24,Billionaires Can't Get Enough of This ETF in 2...,2.126579e-07,1.000000,2.185417e-07
6864,2024-01-24,487.809998,488.769989,484.880005,485.390015,81765000,0.0,488.029999,1,1.000546,...,41.0,1.130333,140.0,1.253644,537.0,2024-01-24,"Stocks May Pause: 5 Reasons to be ""Strategical...",5.955281e-04,0.999267,1.375334e-04


In [22]:
sp500

,Date,Open,High,Low,Close,Volume,Capital Gains,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
0,1997-01-14,47.612693,48.049240,47.496280,47.787312,2111200,0.0,47.670891,0,1.006230,1.0,1.015126,3.0,1.047089,36.0,1.147327,135.0,1.494512,527.0
1,1997-01-15,47.632087,47.932819,47.418664,47.670891,1583900,0.0,47.864925,1,0.998780,1.0,1.007091,3.0,1.043140,35.0,1.143392,134.0,1.489816,526.0
2,1997-01-16,47.826121,47.991039,47.496286,47.864925,1308400,0.0,48.155952,1,1.002031,1.0,1.006487,3.0,1.045890,36.0,1.146884,134.0,1.494815,526.0
3,1997-01-17,47.932829,48.272365,47.874622,48.155952,1604000,0.0,48.214153,1,1.003031,2.0,1.008820,3.0,1.050519,37.0,1.152668,134.0,1.502832,526.0
4,1997-01-20,48.272360,48.485782,48.097741,48.214153,1889900,0.0,48.602196,1,1.000604,2.0,1.005747,4.0,1.050095,37.0,1.152899,134.0,1.503580,526.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6798,2024-01-19,477.649994,482.720001,476.540009,482.429993,110733300,0.0,483.450012,1,1.006194,2.0,1.012309,3.0,1.060597,39.0,1.126278,139.0,1.247738,536.0
6799,2024-01-22,484.010010,485.220001,482.779999,483.450012,75844900,0.0,484.859985,1,1.001056,2.0,1.011575,3.0,1.060449,39.0,1.127724,139.0,1.249808,536.0
6800,2024-01-23,484.010010,485.109985,482.890015,484.859985,49945300,0.0,485.390015,1,1.001456,2.0,1.010327,4.0,1.060866,40.0,1.130057,140.0,1.252861,537.0
6801,2024-01-24,487.809998,488.769989,484.880005,485.390015,81765000,0.0,488.029999,1,1.000546,2.0,1.005940,5.0,1.059148,41.0,1.130333,140.0,1.253644,537.0


In [8]:
# Define additional predictors based on horizons
horizons = [2, 5, 60, 250, 1000]
new_predictors = ["Close", "Volume", "Open", "High", "Low"]  # Original features

for horizon in horizons:
    rolling_average = sp500.rolling(horizon).mean()
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_average["Close"]
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    new_predictors += [ratio_column, trend_column]

DataError: Cannot aggregate non-numeric type: object

In [23]:
# Fill missing sentiment values with 0
combined_data.fillna({'positive': 0, 'neutral': 0, 'negative': 0}, inplace=True)

# Add sentiment scores to predictors
new_predictors += ['positive', 'neutral', 'negative']

# Drop any remaining na values
combined_data = combined_data.dropna()

In [31]:
# Scaling the features
scaler = StandardScaler()
combined_scaled = combined_data.copy()
combined_scaled[new_predictors] = scaler.fit_transform(combined_data[new_predictors])

# Neural Network Model Definition
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Backtest Function
def backtest_nn(data, create_model, predictors, start=10, step=5):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()

        X_train = train[predictors]
        X_test = test[predictors]
        y_train = train["Target"]
        y_test = test["Target"]

        model = create_model(X_train.shape[1])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

        preds = model.predict(X_test)
        preds = (preds >= 0.6).astype(int).flatten()
        predictions = pd.Series(preds, index=test.index, name="Predictions")
        combined = pd.concat([test["Target"], predictions], axis=1)

        all_predictions.append(combined)
    return pd.concat(all_predictions)

# Running the Backtest
predictions_nn = backtest_nn(combined_scaled, create_model, new_predictors)

# Print predictions
print(predictions_nn)

1/1 [==============================] - 0s 57ms/step
      Target  Predictions
6787       0            1
6788       0            1
6790       1            1
6791       1            1
6792       1            1
...      ...          ...
6860       1            1
6861       1            0
6862       1            0
6863       1            0
6864       1            1

[76 rows x 2 columns]


In [21]:
print(f"Number of rows in combined_scaled: {combined_scaled.shape[0]}")

Number of rows in combined_scaled: 86


In [28]:
precision = precision_score(predictions_nn["Target"], predictions_nn["Predictions"])
print(f"Precision: {precision}")

KeyError: 'Target'

In [11]:
print(combined_data.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Capital Gains', 'date', 'title', 'positive', 'neutral',
       'negative'],
      dtype='object')


In [12]:
print(train.columns)

NameError: name 'train' is not defined

In [13]:
print(combined_scaled.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Capital Gains', 'date', 'title', 'positive', 'neutral',
       'negative'],
      dtype='object')


In [33]:
# Join "Predictions" with "Date" from the original combined_data DataFrame
predictions_nn["Date"] = combined_data.loc[predictions_nn.index, "Date"]

# Print the resulting DataFrame
print(predictions_nn[["Date", "Target", "Predictions"]])

            Date  Target  Predictions
6787  2023-12-19       0            1
6788  2023-12-19       0            1
6790  2023-12-21       1            1
6791  2023-12-21       1            1
6792  2023-12-21       1            1
...          ...     ...          ...
6860  2024-01-22       1            1
6861  2024-01-23       1            0
6862  2024-01-23       1            0
6863  2024-01-24       1            0
6864  2024-01-24       1            1

[76 rows x 3 columns]


In [34]:
# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Print the DataFrame with full data
print(predictions_nn)

      Target  Predictions        Date
6787       0            1  2023-12-19
6788       0            1  2023-12-19
6790       1            1  2023-12-21
6791       1            1  2023-12-21
6792       1            1  2023-12-21
6793       1            1  2023-12-21
6794       1            0  2023-12-22
6795       1            0  2023-12-26
6796       1            0  2023-12-26
6797       1            0  2023-12-26
6798       1            1  2023-12-26
6799       1            1  2023-12-26
6800       1            1  2023-12-27
6801       0            1  2023-12-28
6802       0            1  2023-12-28
6803       0            0  2023-12-28
6804       0            0  2023-12-28
6805       0            0  2023-12-29
6806       0            0  2023-12-29
6807       0            0  2023-12-29
6808       0            0  2023-12-29
6809       0            0  2023-12-29
6810       0            0  2023-12-29
6811       0            0  2023-12-29
6812       0            0  2023-12-29
6813       0

In [41]:
# Assuming your DataFrame is named 'predictions_nn'
date_threshold = '2023-12-19'

# Find the index corresponding to the date threshold
date_index = predictions_nn[predictions_nn.index >= date_threshold].index.min()

# Filter the DataFrame using the index
filtered_predictions_nn = predictions_nn.loc[date_index:]

# Print the filtered DataFrame
print(filtered_predictions_nn)

                           Target  Predictions
Date                                          
2023-12-19 00:00:00-05:00       0            0
2023-12-20 00:00:00-05:00       1            0
2023-12-21 00:00:00-05:00       1            0
2023-12-22 00:00:00-05:00       1            0
2023-12-26 00:00:00-05:00       1            0
2023-12-27 00:00:00-05:00       1            0
2023-12-28 00:00:00-05:00       0            0
2023-12-29 00:00:00-05:00       0            0
2024-01-02 00:00:00-05:00       0            0
2024-01-03 00:00:00-05:00       0            0
2024-01-04 00:00:00-05:00       1            0
2024-01-05 00:00:00-05:00       1            1
2024-01-08 00:00:00-05:00       0            0
2024-01-09 00:00:00-05:00       1            1
2024-01-10 00:00:00-05:00       0            0
2024-01-11 00:00:00-05:00       1            0
2024-01-12 00:00:00-05:00       0            0
2024-01-16 00:00:00-05:00       0            0
2024-01-17 00:00:00-05:00       1            0
2024-01-18 00